## Requirements

In [1]:
import numpy as np
import pandas as pd

## Original dataframe

In [39]:
df_orig = pd.read_excel('data/patient_experiment.xlsx',
                        dtype={'dose': np.float32,
                        'temperature': np.float32})

In [40]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   patient      62 non-null     int64         
 1   dose         61 non-null     float32       
 2   date         62 non-null     datetime64[ns]
 3   temperature  61 non-null     float32       
dtypes: datetime64[ns](1), float32(2), int64(1)
memory usage: 1.6 KB


In [41]:
df_orig.head()

,patient,dose,date,temperature
0,1,0.0,2012-10-02 10:00:00,38.299999
1,1,2.0,2012-10-02 11:00:00,38.500000
2,1,2.0,2012-10-02 12:00:00,38.099998
3,1,2.0,2012-10-02 13:00:00,37.299999
4,1,0.0,2012-10-02 14:00:00,37.500000


## To wide format: pivot

In [42]:
df_wide = df_orig.pivot(
    index='date',
    values=['temperature', 'dose'],
    columns=['patient']
)

In [43]:
df_wide.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7 entries, 2012-10-02 10:00:00 to 2012-10-02 16:00:00
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   (temperature, 1)  7 non-null      float32
 1   (temperature, 2)  7 non-null      float32
 2   (temperature, 3)  6 non-null      float32
 3   (temperature, 4)  7 non-null      float32
 4   (temperature, 5)  7 non-null      float32
 5   (temperature, 6)  6 non-null      float32
 6   (temperature, 7)  7 non-null      float32
 7   (temperature, 8)  7 non-null      float32
 8   (temperature, 9)  7 non-null      float32
 9   (dose, 1)         7 non-null      float32
 10  (dose, 2)         7 non-null      float32
 11  (dose, 3)         7 non-null      float32
 12  (dose, 4)         6 non-null      float32
 13  (dose, 5)         7 non-null      float32
 14  (dose, 6)         6 non-null      float32
 15  (dose, 7)         7 non-null      float32
 16  (dose, 8)

Now you have a dataframe with the date as index, and multi-level columns.  The top-level is the temerature and the dose, but next level is the patient ID.

In [44]:
df_wide.head()

temperature                                              \
patient                       1          2          3          4          5   
date                                                                          
2012-10-02 10:00:00   38.299999  39.299999  37.900002  38.099998  37.900002   
2012-10-02 11:00:00   38.500000  39.400002  39.500000  37.200001  39.500000   
2012-10-02 12:00:00   38.099998  38.099998  38.299999  36.099998  38.299999   
2012-10-02 13:00:00   37.299999  37.299999        NaN  35.900002  38.500000   
2012-10-02 14:00:00   37.500000  36.799999  37.700001  36.299999  39.400002   

                                                                dose       \
patient                      6          7          8          9    1    2   
date                                                                        
2012-10-02 10:00:00  37.500000  39.500000  37.799999  38.299999  0.0  0.0   
2012-10-02 11:00:00  38.099998  40.700001  37.900002  39.500000  2.0  5.0   
2012-10-02 12:00:00  37.900002  39.799999  37.400002  40.200001  2.0  5.0   
2012-10-02 13:00:00  37.700001  40.200001  37.599998  39.099998  2.0  5.0   
2012-10-02 14:00:00  37.200001  38.299999  37.299999  37.900002  0.0  0.0   

                                                          
patient                3    4    5    6     7    8     9  
date                                                      
2012-10-02 10:00:00  0.0  0.0  0.0  0.0   0.0  0.0   0.0  
2012-10-02 11:00:00  2.0  5.0  3.0  2.0  10.0  0.0  10.0  
2012-10-02 12:00:00  5.0  5.0  7.0  3.0   5.0  0.0  12.0  
2012-10-02 13:00:00  2.0  0.0  5.0  2.0   8.0  0.0   4.0  
2012-10-02 14:00:00  2.0  NaN  9.0  1.0   3.0  0.0   4.0

## And back again: stack + reset index

In [45]:
df_long = df_wide \
    .stack('patient', future_stack=True)                        \
    .reset_index()   

In [46]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         63 non-null     datetime64[ns]
 1   patient      63 non-null     int64         
 2   temperature  61 non-null     float32       
 3   dose         61 non-null     float32       
dtypes: datetime64[ns](1), float32(2), int64(1)
memory usage: 1.6 KB


In [47]:
df_long.head()

,date,patient,temperature,dose
0,2012-10-02 10:00:00,1,38.299999,0.0
1,2012-10-02 10:00:00,2,39.299999,0.0
2,2012-10-02 10:00:00,3,37.900002,0.0
3,2012-10-02 10:00:00,4,38.099998,0.0
4,2012-10-02 10:00:00,5,37.900002,0.0


Breaking it down into two steps, first the `stack()` method:

In [48]:
df_wide.stack("patient", future_stack=True)

temperature  dose
date                patient                   
2012-10-02 10:00:00 1          38.299999   0.0
                    2          39.299999   0.0
                    3          37.900002   0.0
                    4          38.099998   0.0
                    5          37.900002   0.0
...                                  ...   ...
2012-10-02 16:00:00 5          37.200001   0.0
                    6                NaN   NaN
                    7          37.299999   1.0
                    8          36.799999   0.0
                    9          37.299999   0.0

[63 rows x 2 columns]

As you can see, this has created a dataframe that has only two columns, but a multi-level index.  The top-level index is the date, the sublevel is the patient.

In [51]:
df_long = df_wide.stack("patient", future_stack=True).reset_index()
df_long

,date,patient,temperature,dose
0,2012-10-02 10:00:00,1,38.299999,0.0
1,2012-10-02 10:00:00,2,39.299999,0.0
2,2012-10-02 10:00:00,3,37.900002,0.0
3,2012-10-02 10:00:00,4,38.099998,0.0
4,2012-10-02 10:00:00,5,37.900002,0.0
...,...,...,...,...
58,2012-10-02 16:00:00,5,37.200001,0.0
59,2012-10-02 16:00:00,6,NaN,NaN
60,2012-10-02 16:00:00,7,37.299999,1.0
61,2012-10-02 16:00:00,8,36.799999,0.0


Resetting the index will create columns out of the multi-level index, so one for the date, a second for the patient ID.

If you prefer to get rid of the column name, simply set it to `None`.

In [52]:
df_long.columns.name = None
df_long

,date,patient,temperature,dose
0,2012-10-02 10:00:00,1,38.299999,0.0
1,2012-10-02 10:00:00,2,39.299999,0.0
2,2012-10-02 10:00:00,3,37.900002,0.0
3,2012-10-02 10:00:00,4,38.099998,0.0
4,2012-10-02 10:00:00,5,37.900002,0.0
...,...,...,...,...
58,2012-10-02 16:00:00,5,37.200001,0.0
59,2012-10-02 16:00:00,6,NaN,NaN
60,2012-10-02 16:00:00,7,37.299999,1.0
61,2012-10-02 16:00:00,8,36.799999,0.0


Except for the order of the columns, and the sorting of the rows, you are back to the original data format.